In [1]:
import numpy as np
import matplotlib
import matplotlib.patches as patches
import matplotlib.pyplot as plt


import sys
sys.path += ['Package/']  ### Specify the directory where the Package is
from functions_for_DDCSBM import *
from dynamic_SC import *
from dynamic_SC_BH import *

import warnings
warnings.filterwarnings("ignore")

In [2]:
T = 5

n = 1000
c = 10
theta = np.ones(n)
n_clusters = 2
theta = np.ones(n)
phi = 1

fluctuation = 0/n_clusters
fraction = np.random.normal(1/n_clusters,0/(4*n_clusters), n_clusters)
fraction = fraction/np.sum(fraction)

n_step = 10
eta_v = np.linspace(0.1,0.98,n_step)[::-1]
c_out_v = np.linspace(4,9.9, n_step)
c_in_v = 2*c-c_out_v

In [ ]:
ov_us = np.zeros((n_step,n_step))
ov_adj = np.zeros((n_step,n_step))


for i in range(n_step):

    c_out = c_out_v[i]
    c_in = 2*c-c_out
    C = matrix_C(c_out, c, fluctuation, fraction)

    
    for j in range(n_step):
        
        eta = eta_v[j]
        
        AT, Label = adjacency_matrix_series_DDCSBM(T, C, c, eta, theta,fraction) 
        
        cluster = dynamic_community_detection(AT, eta, real_classes = Label, n_clusters = n_clusters)
        ovA, mod = dynamic_A(AT, n_clusters, Label)
        
        ov_adj[i][j] +=  ovA[-1]
        ov_us[i][j] +=  cluster.overlap[-1]
        
        OUT = 'i : ' + str(i+1) + '; j : ' + str(j+1) 
        sys.stdout.write('\r%s' % OUT)
        sys.stdout.flush()

i : 5; j : 10

In [ ]:
alpha = (c_in_v - c_out_v)/np.sqrt(c)*np.sqrt(phi)/2
x_min = min(alpha)
alpha_c = [find_transition(T,eta_v[i]) for i in range(n_step)]


X,Y = np.meshgrid(alpha,eta_v)

fig = plt.figure(figsize = (14,12))

ax1 = plt.subplot2grid((10,10), (5,0), colspan=5, rowspan=5)
ax2 = plt.subplot2grid((10,10), (5,5), colspan=5, rowspan=5)

fs = 18
ts = 14
levels = np.linspace(0,1,150)
ax1.pcolormesh(X,Y,ov_us.T, cmap = 'Oranges', vmin = 0, vmax = 1)
ax1.set_xlabel(r'$\alpha$', fontsize = fs)
ax1.set_ylabel(r'$\eta$', fontsize = fs)
ax1.tick_params(axis="x", labelsize=ts)
ax1.tick_params(axis="y", labelsize=ts)
ax1.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')
x = np.linspace(x_min,1)
ax1.plot(alpha_c,eta_v, color = 'k', linestyle= '-.', label = r'$\alpha_c(T,\eta)$')
ax1.set_ylim(min(eta_v),max(eta_v))
ax1.legend(fontsize = 12, loc = 3)


ax2.set_xlabel(r'$\alpha$', fontsize = fs)
ax2.tick_params(axis="x", labelsize=ts)
ax2.tick_params(axis="y", labelsize=0)
ax2.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')
im = ax2.pcolormesh(X,Y,ov_adj.T, cmap = 'Oranges', vmin = 0, vmax = 1)
fig.colorbar(im, ax=ax2, fraction=0.046, pad=0.04, ticks = [i*0.2 for i in range(6)]) 
ax2.set_ylim(min(eta_v),max(eta_v))
ax2.plot(alpha_c,eta_v, color = 'k', linestyle= '-.')

plt.show();
